In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Code ein-/ausblenden"></form>''')

In [4]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore',category=DeprecationWarning)
import glob, os
import numpy as np
import gensim
from gensim import corpora, models
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
import yaspin
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import datetime
import nltk
nltk.data.path += [os.getcwd()+"/nltk_data"]

# Kunde Text-Mining showcase
Dieser showcase zeigt, wie wir Text-Mining nutzen können, um über Dokumente zu lernen, Themen zu identifizieren und ähnliche Dokumente zu finden.

**Folgende Probleme werden untersucht:**

**1. Clustern und Finden von ähnlichen Dokumenten**

 **2. Suche von Dokumenten anhand von Suchbegriffen bzw Queries**

Zuerst konvertieren wir die Dokumente von PDF und Docx zu text-Dateien. Anschließend identifizieren wir aus dem rohen Text Sätze, finden einzelne Wörter - sog. Tokens - und entfernen Stopwords ("und","oder" usw.). Außerdem messen wir die Häufigkeit von Wörtern relativ zu dem Dokument, zu dem sie gehören. Das hilft uns später bei der Suche nach Begriffen.

Nach dieser Vorverarbeitung werden trainieren wir ein Machine-Learning-Modell, um automatisiert ähnliche Dokumente in Topics zu organisieren. Zum Schluss zeigen wir, wie sich Ähnlichkeiten von Dokumenten darstellen lassen und präsentieren eine Suche nach ähnlichen Dokumenten anhand eines Suchbegriffs.

In [10]:
f = open(os.getcwd() + '/../data/latest-news-of-month.txt','r')
raw_documents = f.readlines()
print("\n Anzahl geladener Dokumente: {}".format(len(raw_documents)))
f.close()



 Anzahl geladener Dokumente: 2885


# Erstellen eines Corpus und des LDA Modells
Nun erstellen wir aus den Dokumenten einen Corpus. Dieser Corpus misst für jedes Wort die Häufigkeit. Um maschinen-lesbar zu sein, wird jedem Wort eine ID zugeordnet, denn mit rohen Worten kann er wenig anfangen und das ist auch irrelevant für das Machine Learning Modell.
Mehr dazu auch hier: [https://machinelearningmastery.com/gentle-introduction-bag-words-model/](https://machinelearningmastery.com/gentle-introduction-bag-words-model/)

Was ein solcher Corpus nicht beachtet, ist die Reihenfolge der Wörter sowie semantische Strukturen!

Zum Schluss trainieren wir ein LDA-Modell. Mit dieser Methode identifizieren wir anhand der Wörter im Corpus automatisiert eine vorgegebene Anzahl Topics. Wir nutzen dieses Modell später, um ähnliche Dokumente zu finden sowie die Topics pro Dokument zu finden.

In [13]:
dictionary = corpora.Dictionary([" ".join(raw_documents).split()]) 

#creating the bag of words object
bow_corpus = [dictionary.doc2bow(text.split()) for text in raw_documents]

total_topics = 3
    
lda_model_bow = models.LdaModel(corpus=bow_corpus,alpha='auto', id2word=dictionary, num_topics=total_topics,
                                passes=5, random_state=47)

print("Training abgeschlossen!")
print('{} verschiedene Wörter im Corpus'.format(len(dictionary)))

Training abgeschlossen!
23984 verschiedene Wörter im Corpus


## Visualisierung der Topics
Nun können wir die identifizierten Topics visualisieren. Auf dem linkem Teil sehen wir die Topics. Die Größe der Blase zeigt, wie viele der Wörter das jeweilige Topic abdeckt.

Auf der rechten Seite sehen wir die Top-30 *relevantesten* Wörter des jeweiligen Topics. 
[Sievert and Shirley (2014)](https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf) definieren Relevanz. Einfach gesagt misst sie zum Einen die Anzahl des Auftauchens des jeweiligen Wortes im identifizierten Topic gegenüber der Wahrscheinlichkeit des Auftretens des Wortes im gesamten Korpus. Der Parameter Lambda gewichtet die Kritieren. Der Sinn der Relevanz ist das Verständlich-Machen von Machine Learning-Ergebnissen. Meist können Menschen nämlich intuitiv nichts mit den Ergebnissen anfangen und Wort-Häufigkeiten alleine reichen nicht, um ein gefundenes Topic erfassen zu können.

Die Formel lautet:

$$ 
\begin{align}
&& r(w,k | \lambda) = \lambda \: log(\phi_{kw}) +\: (1 - \lambda) \: log(\frac{\phi_{kw}}{p_w}) \\ 
&& w \in Vokabular \\
&& k \in Topics\\
&& w,k \in \mathbb{N} \\
\end{align}
$$

Wenn Lambda = 1, werden die Wörter einfach nach Häufigkeit sortiert.
Ein empirisch gefundener, funktionierender Wert für Lambda ist 0.6. Er erlaubt den meisten Nutzern eine klare Abgrenzung von Topics anhand der 30 relevantesten Worte.

In [12]:
data = pyLDAvis.gensim.prepare(lda_model_bow, bow_corpus, dictionary)

data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.148632 -0.050107       1        1  28.824806
0     -0.178293 -0.066607       2        1  18.551222
5      0.169550  0.018793       3        1  15.745969
1      0.120868  0.024744       4        1  12.986837
3     -0.125237 -0.066925       5        1  12.816182
4     -0.135520  0.140102       6        1  11.074987, topic_info=      Category         Freq       Term        Total  loglift  logprob
term                                                                 
22386  Default  1094.000000        the  1094.000000  30.0000  30.0000
22477  Default   827.000000         to   827.000000  29.0000  29.0000
15982  Default   660.000000        der   660.000000  28.0000  28.0000
16054  Default   620.000000        die   620.000000  27.0000  27.0000
13884  Default   657.000000          a   657.000000  26.0000  26.0000
19971  Default   515.000000         of   515.000000  25.0000  25.0000
22730  Default   744.000000        und   744.000000  24.0000  24.0000
14213  Default   517.000000        and   517.000000  23.0000  23.0000
17434  Default   374.000000        für   374.000000  22.0000  22.0000
15871  Default   103.000000         de   103.000000  21.0000  21.0000
15960  Default   341.000000        den   341.000000  20.0000  20.0000
23238  Default   273.000000        von   273.000000  19.0000  19.0000
19597  Default   357.000000        mit   357.000000  18.0000  18.0000
18643  Default   279.000000         is   279.000000  17.0000  17.0000
5309   Default   112.000000     Formel   112.000000  16.0000  16.0000
14427  Default   244.000000        auf   244.000000  15.0000  15.0000
12062  Default   312.000000        The   312.000000  14.0000  14.0000
20013  Default   259.000000         on   259.000000  13.0000  13.0000
717    Default    90.000000         1:    90.000000  12.0000  12.0000
17264  Default   298.000000        for   298.000000  11.0000  11.0000
23767  Default   186.000000         zu   186.000000  10.0000  10.0000
12961  Default    65.000000     Vettel    65.000000   9.0000   9.0000
14391  Default   214.000000         as   214.000000   8.0000   8.0000
21609  Default   190.000000       sich   190.000000   7.0000   7.0000
23603  Default   207.000000       with   207.000000   6.0000   6.0000
15844  Default   206.000000        das   206.000000   5.0000   5.0000
16543  Default    55.000000         en    55.000000   4.0000   4.0000
3927   Default   250.000000        Die   250.000000   3.0000   3.0000
18398  Default   312.000000         im   312.000000   2.0000   2.0000
6143   Default    54.000000   Hamilton    54.000000   1.0000   1.0000
...        ...          ...        ...          ...      ...      ...
3603    Topic6     5.246341      Cupra     5.927670   2.0784  -7.4848
13484   Topic6    10.578971    Windows    13.014996   1.9932  -6.7835
14984   Topic6    28.223862        bis    47.586555   1.6781  -5.8022
17270   Topic6     8.681774    fordert    10.949321   1.9684  -6.9811
8412    Topic6    21.774916  Millionen    36.511379   1.6836  -6.0616
4348    Topic6     7.056784       Echo     8.491463   2.0154  -7.1883
23238   Topic6    88.346458        von   273.876740   1.0691  -4.6611
4727    Topic6    17.571367       Euro    30.837654   1.6380  -6.2760
23767   Topic6    59.255184         zu   186.157471   1.0557  -5.0605
15982   Topic6   121.078484        der   660.130920   0.5045  -4.3459
3927    Topic6    61.102848        Die   250.839813   0.7882  -5.0298
17434   Topic6    77.943130        für   374.709656   0.6303  -4.7863
16054   Topic6    93.981369        die   620.209290   0.3135  -4.5992
14419   Topic6    31.563969       auch   113.226196   0.9231  -5.6903
22730   Topic6    83.957787        und   744.775513   0.0177  -4.7120
3858    Topic6    38.000702        Der   169.586761   0.7047  -5.5047
14427   Topic6    43.440094        auf   244.593018   0.4723  -5.3709
18398   Topic6    43.9417

## Zuordnung der Dokumente zu Topics
Natürlich können wir auch für jedes Dokument sehen, welche Topics es enthält. 
Mit dieser Ansicht können wir den Algorithmus auch tunen oder uns für mehr/weniger Topics entscheiden.

NameError: name 'files' is not defined

## Suche nach Dokumenten mittels Wörtern
Natürlich möchten wir auch anhand von Wörtern ähnliche Dokumente finden. Dazu nutzen wir die anfangs berechneten Gewichte jeden Wortes pro Dokument.

Um auch verwandte Wörter zu dem Suchwort zu finden, nutzen wir die [Levenshtein-Distanz](https://de.wikipedia.org/wiki/Levenshtein-Distanz). Diese misst die Distanz von Wörtern anhand von der Anzahl Bearbeitungsschritte, um Wort A in Wort B zu überführen. Zum Beispiel ist die Distanz der Wörter "Car" und "Cart" gleich 1, da ein Buchstabe hinzugefügt werden muss, also eine Bearbeitung notwendig ist, um das Wort in das andere zu überführen.

In diesem Showcase lässt sich exakt ein Begriff suchen. Als Antwort erhalten Sie eine Tabelle mit Dateien und welche (ähnlichen) Wörter sie enthalten. Die prozentuale Häufigkeit im jewiligen Dokument dient als Orientierung bzgl. der Relevanz des Suchwortes im Dokument.

Im aktuellen Beispiel werden Worte gesucht, die maximal 2 Bearbeitungsschritte benötigen. Pro Dokument werden bis zu 10 Ergebnisse sortiert nach ihrem Gewicht im jeweiligen Dokument ausgegeben.

In [8]:

out = widgets.Output()

from IPython.display import clear_output
BOLD = '\033[1m' 
NORMAL = '\033[0m'

def search(b):
    searchResult = wordSearchByDoc(textbox.value,freqs,threshold=2,out=out)


    with out:
        #print("Wörter in {}  Dokumenten gefunden! \n".format(len(searchResult.keys())))
        print("{:<75} {:<35} {:<55}".format(BOLD+"Dokument","Wort", "Häufigkeit im Dokument(%)"+NORMAL))
    i=0

    for word, values in  searchResult.items():

        for results in values:
            if i <= 10:
                if i == 0:
                    output_string = "{:<75} {:<35} {:<55}".format(word,results[0].replace(".txt","").replace("tmp/",""), str(round(results[1]*100,2))+"%")
                    
                    with out:
                        print("_"* len(output_string))
                        print(output_string)

                else:
                    with out:
                        print("{:<75} {:<35} {:<55}".format("",results[0].replace(".txt","").replace("tmp/",""), str(round(results[1]*100,2))+"%"))

                i+=1
        i = 0
        
    return searchResult

      
textbox = widgets.Text(value="Levenshtein",placeholder='Suchbegriff eingeben',description='Suche:',disabled=False)
button= widgets.Button(description="Suchen")
button.on_click(search)
widgets.VBox([widgets.HBox([textbox,button]),out])


## Suche mit Whoosh
[Whoosh](https://whoosh.readthedocs.io/en/latest/) ist eine offene Bibliothek, mit der wir eine Suchmaschine schnell und problemlos anlegen können. Sie kann sehr viel komplexere Dinge als eine einfache Suche mit der Levenshtein-Distanz. Komplexe Suchanfragen sind ebenfalls möglich, oder die Autovervollständigung mittels sogenannter n-Gramme.

In diesem einfachen Beispiel werden bereits Vorschläge gemacht, wenn es kein Ergebnis gibt. Komplexere Suchen sind mittels einfacher API möglich. Mittels moderner Technologien wie ElasticSearch lassen sich ebenfalls sehr schnelle und effiziente Suchen implementieren!

In [9]:
from whoosh.fields import Schema, TEXT
from whoosh.index import open_dir
from yaspin import yaspin

import os.path
from whoosh.index import create_in

## ERSTELLEN DES INDEX
schema = Schema(title=TEXT(stored=True), content=TEXT(stored=True))
if not os.path.exists("index"):
    os.mkdir("index")
ix = create_in("index", schema)

ix = open_dir("index")

writer = ix.writer()
for index, file in files.items():
    writer.add_document(title=file, content=raw_documents[index])
writer.commit()

searcher = ix.searcher()
out_whoosh = Output()

def search_whoosh(b):
    with out_whoosh:
        clear_output()
        with yaspin(text="Loading", color="yellow") as spinner:
            spinner.start()
    
            querystring = textbox_whoosh.value
            from whoosh.qparser import QueryParser
            parser = QueryParser("content", ix.schema)
            myquery = parser.parse(querystring)
            results = searcher.search(myquery)
            spinner.stop()
            clear_output()
            
    
            if len(results) >0:
                print("Hit in folgenden Dateien:")
                for result in results:
                    print(result["title"].replace("tmp/","").replace(".txt",""))
                    
            else:
                mistyped_words= textbox_whoosh.value.split()
                with ix.searcher() as s:
                    corrector = s.corrector("content")
                    print("Keine Ergebnisse. Meinten Sie vielleicht eines dieser Wörter:")
                    suggestions = []
                    for mistyped_word in mistyped_words:
                        suggestions += corrector.suggest(mistyped_word, limit=3)
                    for suggestion in suggestions:
                        print(suggestion)

                
## Visualization
textbox_whoosh = widgets.Text(value="Whoosh",placeholder='Suchbegriff eingeben',description='Suche:',disabled=False)
button_whoosh= widgets.Button(description="Suchen")
button_whoosh.on_click(search_whoosh)
widgets.VBox([widgets.HBox([textbox_whoosh,button_whoosh]),out_whoosh])